In [0]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# imports
from math import factorial
from collections import Counter
from itertools import combinations, islice
from scipy.stats import geom,poisson,binom,gamma
from scipy import stats
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set()
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [0]:
pidfilename = '/content/drive/Shared drives/ML Project/not_imp_csvs/pidBK7610.csv'
pid_df = pd.read_csv(pidfilename)

In [0]:
pid_df.head(5)

,time,pid,x,y,z
0,1493735870653,BK7610,0.1261,-0.0078,-0.0243
1,1493735870679,BK7610,0.1336,-0.0697,-0.0446
2,1493735870703,BK7610,0.1443,-0.0474,-0.0447
3,1493735870729,BK7610,0.1255,-0.0038,0.0111
4,1493735870753,BK7610,0.1076,0.0032,0.0276


In [0]:
pid_df = pid_df[pid_df['time']!=0]

In [0]:
len(pid_df)

1225727

In [0]:
one_second_list= []
# list_index.append(2)
# st_time = df1['time'][2]
# old_pid = df1['pid'][2]
start_index = pid_df.index[0]
# print(start_index)
old_dig = list(str(pid_df['time'][start_index]))[-4]

# LIMIT = 100

for row in (pid_df.iterrows()):
  print(row[0])
  new_dig = list(str(row[1][0]))[-4]
  if new_dig != old_dig:
    one_second_list.append(row[0])
    old_dig = new_dig

# print(one_second_list)

In [0]:
with open("listBK7610.txt", "w") as f:
    for o in one_second_list:
        f.write(str(o) +"\n")

In [0]:
#Generating features

columns = ['time','pid','x','y','z']
df1 = pd.DataFrame(columns=columns)

cur_pos = pid_df.index[0]
cur_pos4 = pid_df.index[0]
# df1 = pid_df.copy()


#1 second statistics
df1['xmean']= df1['ymean'] = df1['zmean'] = 0 
df1['xstd']= df1['ystd'] = df1['zstd'] = 0
df1['xmed']= df1['ymed'] = df1['zmed'] = 0
df1['xzcr']= df1['yzcr'] = df1['zzcr'] = 0 #zero crossing rate
df1['xmaxr'] = df1['ymaxr'] = df1['zmaxr'] = 0
df1['xminr'] = df1['yminr'] = df1['zminr'] = 0

# 6 summary statistics
#4 second means
df1['xmean_mean_4']= df1['ymean_mean_4'] = df1['zmean_mean_4'] = 0
df1['xstd_mean_4']= df1['ystd_mean_4'] = df1['zstd_mean_4'] = 0
df1['xmed_mean_4']= df1['ymed_mean_4'] = df1['zmed_mean_4'] = 0
df1['xzcr_mean_4'] = df1['yzcr_mean_4'] = df1['zzcr_mean_4'] = 0

#4 second variances
df1['xmean_var_4']= df1['ymean_var_4'] = df1['zmean_var_4'] = 0
df1['xstd_var_4']= df1['ystd_var_4'] = df1['zstd_var_4'] = 0
df1['xmed_var_4']= df1['ymed_var_4'] = df1['zmed_var_4'] = 0
df1['xzcr_var_4'] = df1['yzcr_var_4'] = df1['zzcr_var_4'] = 0

#4 second max
df1['xmean_max_4']= df1['ymean_max_4'] = df1['zmean_max_4'] = 0
df1['xstd_max_4']= df1['ystd_max_4'] = df1['zstd_max_4'] = 0
df1['xmed_max_4']= df1['ymed_max_4'] = df1['zmed_max_4'] = 0
df1['xzcr_max_4'] = df1['yzcr_max_4'] = df1['zzcr_max_4'] = 0

#4 second min
df1['xmean_min_4']= df1['ymean_min_4'] = df1['zmean_min_4'] = 0
df1['xstd_min_4']= df1['ystd_min_4'] = df1['zstd_min_4'] = 0
df1['xmed_min_4']= df1['ymed_min_4'] = df1['zmed_min_4'] = 0
df1['xzcr_min_4'] = df1['yzcr_min_4'] = df1['zzcr_min_4'] = 0

#4 second mean_lower_third
df1['xmean_mean_lt_4']= df1['ymean_mean_lt_4'] = df1['zmean_mean_lt_4'] = 0
df1['xstd_mean_lt_4']= df1['ystd_mean_lt_4'] = df1['zstd_mean_lt_4'] = 0
df1['xmed_mean_lt_4']= df1['ymed_mean_lt_4'] = df1['zmed_mean_lt_4'] = 0
df1['xzcr_mean_lt_4'] = df1['yzcr_mean_lt_4'] = df1['zzcr_mean_lt_4'] = 0

#4 second mean_upper_third
df1['xmean_mean_ut_4']= df1['ymean_mean_ut_4'] = df1['zmean_mean_ut_4'] = 0
df1['xstd_mean_ut_4']= df1['ystd_mean_ut_4'] = df1['zstd_mean_ut_4'] = 0
df1['xmed_mean_ut_4']= df1['ymed_mean_ut_4'] = df1['zmed_mean_ut_4'] = 0
df1['xzcr_mean_ut_4'] = df1['yzcr_mean_ut_4'] = df1['zzcr_mean_ut_4'] = 0

fours = 0
new_index = 0
new_index_4 = 0

for i,l in enumerate(one_second_list):
  window = pid_df[cur_pos:l]
  fours += 1

  #assign basic features
  df1.at[new_index,'time'] = int(str(pid_df['time'][cur_pos])[:-3])
  df1.at[new_index,'x'] = pid_df['x'][cur_pos]
  df1.at[new_index,'y'] = pid_df['y'][cur_pos]
  df1.at[new_index,'z'] = pid_df['z'][cur_pos]
  df1.at[new_index,'pid'] = pid_df['pid'][cur_pos]
 

  # extract features for 1 second
  # means
  f_means = np.mean(window[['x', 'y', 'z']])
  # print(f_means)
  df1.at[new_index, 'xmean'] = f_means[0]
  df1.at[new_index, 'ymean'] = f_means[1]
  df1.at[new_index, 'zmean'] = f_means[2]
  
  # std_dev
  f_stds = np.std(window[['x','y','z']])
  df1.at[new_index,'xstd'] = f_stds[0]
  df1.at[new_index,'ystd'] = f_stds[1]
  df1.at[new_index,'zstd'] = f_stds[2]

  # medians
  f_meds = np.median(window[['x','y','z']], axis=0)
  df1.at[new_index,'xmed'] = f_meds[0]
  df1.at[new_index,'ymed'] = f_meds[1]
  df1.at[new_index,'zmed'] = f_meds[2]

  # zcr
  f_zcr_x = len(np.where(np.diff(np.sign(window['x'])))[0])
  f_zcr_y = len(np.where(np.diff(np.sign(window['y'])))[0])
  f_zcr_z = len(np.where(np.diff(np.sign(window['z'])))[0])
  df1.at[new_index,'xzcr'] = f_zcr_x
  df1.at[new_index,'yzcr'] = f_zcr_y
  df1.at[new_index,'zzcr'] = f_zcr_z

  #max of raw signal
  f_maxr = np.max(window[['x','y','z']], axis=0)
  df1.at[new_index,'xmaxr'] = f_maxr[0]
  df1.at[new_index,'ymaxr'] = f_maxr[1]
  df1.at[new_index,'zmaxr'] = f_maxr[2]
  
  #min of raw signal
  f_minr = np.min(window[['x','y','z']], axis=0)
  df1.at[new_index,'xminr'] = f_minr[0]
  df1.at[new_index,'yminr'] = f_minr[1]
  df1.at[new_index,'zminr'] = f_minr[2]


  new_index += 1
  # extract features for 4 seconds
  if fours == 10 or i == len(one_second_list)-1:
    window = df1[new_index_4:new_index]
    df1.at[new_index,'time'] = int(str(pid_df['time'][cur_pos])[:-3])
    df1.at[new_index,'x'] = pid_df['x'][cur_pos]
    df1.at[new_index,'y'] = pid_df['y'][cur_pos]
    df1.at[new_index,'z'] = pid_df['z'][cur_pos]
    df1.at[new_index,'pid'] = pid_df['pid'][cur_pos]
 
    # means
    f_means = np.mean(window[['xmean', 'ymean', 'zmean']])
    df1.at[new_index_4:new_index, 'xmean_mean_4'] = f_means[0]
    df1.at[new_index_4:new_index,'ymean_mean_4'] = f_means[1]
    df1.at[new_index_4:new_index,'zmean_mean_4'] = f_means[2]

    f_means = np.mean(window[['xstd', 'ystd', 'zstd']])
    df1.at[new_index_4:new_index, 'xstd_mean_4'] = f_means[0]
    df1.at[new_index_4:new_index, 'ystd_mean_4'] = f_means[1]
    df1.at[new_index_4:new_index, 'zstd_mean_4'] = f_means[2]

    f_means = np.mean(window[['xmed', 'ymed', 'zmed']])
    df1.at[new_index_4:new_index, 'xmed_mean_4'] = f_means[0]
    df1.at[new_index_4:new_index, 'ymed_mean_4'] = f_means[1]
    df1.at[new_index_4:new_index, 'zmed_mean_4'] = f_means[2]

    f_means = np.mean(window[['xzcr', 'yzcr', 'zzcr']])
    df1.at[new_index_4:new_index, 'xzcr_mean_4'] = f_means[0]
    df1.at[new_index_4:new_index, 'yzcr_mean_4'] = f_means[1]
    df1.at[new_index_4:new_index, 'zzcr_mean_4'] = f_means[2]


    # variances
    f_vars = np.var(window[['xmean', 'ymean', 'zmean']])
    df1.at[new_index_4:new_index, 'xmean_var_4'] = f_vars[0]
    df1.at[new_index_4:new_index, 'ymean_var_4'] = f_vars[1]
    df1.at[new_index_4:new_index, 'zmean_var_4'] = f_vars[2]

    f_vars = np.var(window[['xstd', 'ystd', 'zstd']])
    df1.at[new_index_4:new_index, 'xstd_var_4'] = f_vars[0]
    df1.at[new_index_4:new_index, 'ystd_var_4'] = f_vars[1]
    df1.at[new_index_4:new_index, 'zstd_var_4'] = f_vars[2]

    f_vars = np.var(window[['xmed', 'ymed', 'zmed']])
    df1.at[new_index_4:new_index, 'xmed_var_4'] = f_vars[0]
    df1.at[new_index_4:new_index, 'ymed_var_4'] = f_vars[1]
    df1.at[new_index_4:new_index, 'zmed_var_4'] = f_vars[2]

    f_vars = np.var(window[['xzcr', 'yzcr', 'zzcr']])
    df1.at[new_index_4:new_index, 'xzcr_var_4'] = f_vars[0]
    df1.at[new_index_4:new_index, 'yzcr_var_4'] = f_vars[1]
    df1.at[new_index_4:new_index, 'zzcr_var_4'] = f_vars[2]


    #max
    f_max = np.max(window[['xmean', 'ymean', 'zmean']], axis = 0)
    df1.at[new_index_4:new_index, 'xmean_max_4'] = f_max[0]
    df1.at[new_index_4:new_index, 'ymean_max_4'] = f_max[1]
    df1.at[new_index_4:new_index, 'zmean_max_4'] = f_max[2]

    f_max = np.max(window[['xstd', 'ystd', 'zstd']], axis = 0)
    df1.at[new_index_4:new_index, 'xstd_max_4'] = f_max[0]
    df1.at[new_index_4:new_index, 'ystd_max_4'] = f_max[1]
    df1.at[new_index_4:new_index, 'zstd_max_4'] = f_max[2]

    f_max = np.max(window[['xmed', 'ymed', 'zmed']], axis = 0)
    df1.at[new_index_4:new_index, 'xmed_max_4'] = f_max[0]
    df1.at[new_index_4:new_index, 'ymed_max_4'] = f_max[1]
    df1.at[new_index_4:new_index, 'zmed_max_4'] = f_max[2]

    f_max = np.max(window[['xzcr', 'yzcr', 'zzcr']], axis = 0)
    df1.at[new_index_4:new_index, 'xzcr_max_4'] = f_max[0]
    df1.at[new_index_4:new_index, 'yzcr_max_4'] = f_max[1]
    df1.at[new_index_4:new_index, 'zzcr_max_4'] = f_max[2]

    
    #min
    f_min = np.min(window[['xmean', 'ymean', 'zmean']], axis = 0)
    df1.at[new_index_4:new_index, 'xmean_min_4'] = f_min[0]
    df1.at[new_index_4:new_index, 'ymean_min_4'] = f_min[1]
    df1.at[new_index_4:new_index, 'zmean_min_4'] = f_min[2]

    f_min = np.min(window[['xstd', 'ystd', 'zstd']], axis = 0)
    df1.at[new_index_4:new_index, 'xstd_min_4'] = f_min[0]
    df1.at[new_index_4:new_index, 'ystd_min_4'] = f_min[1]
    df1.at[new_index_4:new_index, 'zstd_min_4'] = f_min[2]

    f_min = np.min(window[['xmed', 'ymed', 'zmed']], axis = 0)
    df1.at[new_index_4:new_index, 'xmed_min_4'] = f_min[0]
    df1.at[new_index_4:new_index, 'ymed_min_4'] = f_min[1]
    df1.at[new_index_4:new_index, 'zmed_min_4'] = f_min[2]

    f_min = np.min(window[['xzcr', 'yzcr', 'zzcr']], axis = 0)
    df1.at[new_index_4:new_index, 'xzcr_min_4'] = f_min[0]
    df1.at[new_index_4:new_index, 'yzcr_min_4'] = f_min[1]
    df1.at[new_index_4:new_index, 'zzcr_min_4'] = f_min[2]


    ###mean lower third ####
    length = 10
    third_length = int(length / 3)

    #mean
    win = window[['xmean']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'xmean_mean_lt_4'] = sorted_win

    win = window[['ymean']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'ymean_mean_lt_4'] = sorted_win

    win = window[['zmean']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'zmean_mean_lt_4'] = sorted_win

    #stddev
    win = window[['xstd']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'xstd_mean_lt_4'] = sorted_win

    win = window[['ystd']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'ystd_mean_lt_4'] = sorted_win

    win = window[['zstd']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'zstd_mean_lt_4'] = sorted_win

    #median
    win = window[['xmed']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'xmed_mean_lt_4'] = sorted_win

    win = window[['ymed']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'ymed_mean_lt_4'] = sorted_win

    win = window[['zmed']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'zmed_mean_lt_4'] = sorted_win
    
    #zcr
    win = window[['xzcr']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'xzcr_mean_lt_4'] = sorted_win

    win = window[['yzcr']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'yzcr_mean_lt_4'] = sorted_win

    win = window[['zzcr']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'zzcr_mean_lt_4'] = sorted_win


    ###mean upper third ####

    #mean
    win = window[['xmean']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'xmean_mean_ut_4'] = sorted_win

    win = window[['ymean']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'ymean_mean_ut_4'] = sorted_win

    win = window[['zmean']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'zmean_mean_ut_4'] = sorted_win

    #stddev
    win = window[['xstd']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'xstd_mean_ut_4'] = sorted_win

    win = window[['ystd']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'ystd_mean_ut_4'] = sorted_win

    win = window[['zstd']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'zstd_mean_ut_4'] = sorted_win

    #median
    win = window[['xmed']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'xmed_mean_ut_4'] = sorted_win

    win = window[['ymed']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'ymed_mean_ut_4'] = sorted_win

    win = window[['zmed']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'zmed_mean_ut_4'] = sorted_win

    #zcr
    win = window[['xzcr']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'xzcr_mean_ut_4'] = sorted_win

    win = window[['yzcr']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'yzcr_mean_ut_4'] = sorted_win

    win = window[['zzcr']]
    sorted_win = np.sort(win, axis = 0)
    sorted_win = sorted_win[::-1]
    sorted_win = np.mean(sorted_win[:third_length])
    df1.at[new_index_4:new_index, 'zzcr_mean_ut_4'] = sorted_win


    fours = 0
    cur_pos4 = l
    new_index_4 += 10


  cur_pos = l
  


  if i % 1000 == 0:
    print(i)
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000


In [0]:
name = "/content/drive/Shared drives/ML Project/pid_csv/pidBK7610"
df1.to_csv(name, index=False)